In [1]:
import numpy as np, pandas as pd, os, gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.special import softmax

from rgf.sklearn import RGFClassifier, FastRGFClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
import warnings

import sys
sys.path.append('/home/raynoldng/Desktop/ieee-fraud-detection')
sys.path.append('/home/raynoldng/Desktop/ieee-fraud-detection/fraud')

In [2]:
train = pd.read_pickle('../processed_input/X_train_encoding_deviceinfo_email.pkl')
test = pd.read_pickle('../processed_input/X_test_encoding_deviceinfo_email.pkl')
y_train = pd.read_pickle('../input/y_test.pkl')

y_train.index = train.index

In [3]:
# need to convert Category types to ints
for f in train.columns:
    if str(train[f].dtype) == 'category':
        train[f] = train[f].cat.codes
        test[f] = test[f].cat.codes

train = train.fillna(-999)
test = test.fillna(-999)

In [7]:
best_params = {
    'l1': 186.4,
    'l2': 5582.0,
    'learning_rate': 0.628,
    'max_depth': 7,
    'max_leaf': 1292,
    'min_samples_leaf': 39
}
model = FastRGFClassifier(
            max_leaf=best_params['max_leaf'],
            max_depth=best_params['max_depth'],
            l1=best_params['l1'],
            l2=best_params['l2'],
            min_samples_leaf=best_params['min_samples_leaf'],
            learning_rate=best_params['learning_rate'],
        )
model.fit(train, y_train)

FastRGFClassifier(calc_prob='sigmoid', data_l2=2.0, l1=186.4, l2=5582.0,
         learning_rate=0.628, loss='LS', max_bin=None, max_depth=7,
         max_leaf=1292, min_child_weight=5.0, min_samples_leaf=39,
         n_estimators=500, n_jobs=-1, opt_algorithm='rgf',
         sparse_max_features=80000, sparse_min_occurences=5,
         tree_gain_ratio=1.0, verbose=0)

In [35]:
def create_submission(model, X_test, name="submit.csv"):
    predict = model.predict_proba(X_test)[:, 1]
    predict[predict > 1] = 1
    predict[predict < 0] = 0
    sample_submission = pd.read_csv('../input/sample_submission.csv')
    submission = pd.DataFrame({
        'TransactionID': sample_submission['TransactionID'],
        'isFraud': predict
    })
    submission.to_csv(name, index=False, float_format='%.3f')

In [36]:
create_submission(model, test, name='rgf_submit.csv')

In [39]:
# save training predictions
train_pred = model.predict_proba(train)

In [41]:
train_submission = pd.DataFrame({
        'TransactionID': train.index,
        'isFraud': train_pred[:, 1]
    })
train_submission.to_csv('rgf-train.csv', index=False, float_format='%.3f')